# 4bit quantization으로 Polyglot-ko 12.8B QLoRA 학습하기

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

- Colab Free (T4 GPU)에서, 한국어 언어 모델 중 가장 큰 크기 모델인 Polyglot-ko 12.8B 모델을 QLoRA로 파인튜닝 해 봅시다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/MyDrive/DACON

/content/drive/MyDrive/DACON


In [4]:
ls

baseline_submit.csv  hansoldeco_polyglot_5.8b/              polyglot_ko_5.8b_outputs/  test.csv
DACON.ipynb          polyglot_12.8b_finetuned_submit_3.csv  prediction.txt             train.csv
formatted_data.txt   polyglot_5.8b_finetuned_submit.csv     sample_submission.csv


In [5]:
import torch
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
!nvidia-smi

Tue Feb 27 04:38:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00


## 데이터셋


In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [9]:
!pip install datasets

In [ ]:
from datasets import Dataset
# max_seq_length = 512
# 데이터 로드
data = pd.read_csv('train.csv')

# 데이터 포맷팅 및 토크나이징
formatted_data = []
for _, row in tqdm(data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            # 질문과 답변 쌍을 </s> token으로 연결
            # input_text = row[q_col] + tokenizer.eos_token + row[a_col]
            # input_ids = tokenizer.encode(input_text,return_tensors='pt')
            # # input_ids = input_ids[:, :max_seq_length]
            input_ids={}
            input_ids['instruction'] =row[q_col]
            input_ids['output'] = row[a_col]
            formatted_data.append(input_ids)
data = Dataset.from_list(formatted_data)

print('Done.')

644it [00:00, 2712.88it/s]

Done.


In [ ]:
data

Dataset({
    features: ['instruction', 'output'],
    num_rows: 6440
})

In [ ]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/6440 [00:00<?, ? examples/s]

## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-5.8b"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00011-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

model-00012-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/515M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## 텍스트 데이터만 tokenize

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/6440 [00:00<?, ? examples/s]

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3670016 || all params: 3070156800 || trainable%: 0.11953838970048696


In [ ]:
!nvidia-smi

Sat Feb 24 14:28:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0              27W /  70W |   4473MiB / 15360MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 학습하기!

- 이번 예제에서는 22k개의 아주아주 일부분인 100개 스텝만 학습해봅시다.

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        num_train_epochs = 2,
        # max_steps=2000, ## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        learning_rate=5e-5,
        fp16=True,
        logging_steps=50,
        output_dir="polyglot_ko_5.8b_outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
50,2.258300
100,1.744100
150,1.688500
200,1.639200
250,1.580500
300,1.541800
350,1.560800
400,1.565900
450,1.541100
500,1.559500


Checkpoint destination directory polyglot_ko_5.8b_outputs/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory polyglot_ko_5.8b_outputs/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be upda

TrainOutput(global_step=6440, training_loss=1.2413192926726726, metrics={'train_runtime': 10569.1445, 'train_samples_per_second': 1.219, 'train_steps_per_second': 0.609, 'total_flos': 6.202935373765018e+16, 'train_loss': 1.2413192926726726, 'epoch': 2.0})

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1181: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1482: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[   6,    6,    6, 2438,   29, 1832, 4770,  272,   34,  202,  202,    6,
            6,    6, 4253,   29, 1832,  296, 1370,  270]])

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
trainer.save_model("./hansoldeco_polyglot_5.8b")

## 몇 가지 팁

- 만약 학습이 충분히 되지 않으면 `<|endoftext|>` 토큰이 잘 생성되지 않을 수 있습니다
- 이럴떈 충분히 긴 `max_new_tokens`를 준 뒤, `###`으로 잘라서 써보세요. ex) `output.split('###')[0]`
- 아래 결과는 실제 위 학습된(50step, 100개 샘플) 모델의 결과물입니다.
- 생성시에 속도가 꽤 느립니다. 1-2tokens/s 정도라 256토큰 생성시 약 ~3분 시간이 소요됩니다.

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-5.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [27]:
from peft import PeftModel

output_dir = "./hansoldeco_polyglot_5.8b" # safetensors 컨버팅된 레포
peft_model = PeftModel.from_pretrained(model, output_dir)
model = peft_model.merge_and_unload()


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:248: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [16]:
import pandas as pd
from tqdm import tqdm

In [17]:
#데이터 분리하는 함수
def split_sentences_with_keywords(sentences, keywords):
    process_x_list = []
    x_index = []

    for i, sentence in enumerate(sentences):
        split_flag = False
        for keyword in keywords:
            if keyword in sentence:
                sentences_split = sentence.split(keyword)
                for sent in sentences_split:
                    process_x_list.append(sent.strip())
                    x_index.append(i)
                split_flag = True
                break
        if not split_flag:
            process_x_list.append(sentence)
            x_index.append(i)

    return  x_index, process_x_list

In [18]:
# 답변을 다시 합치는 함수
def merge_answers_by_index(indexes, strings):
    output = []
    temp_str = ''

    for index, string in zip(indexes, strings):
        if output and output[-1][0] == index:
            output[-1][1] += ' 그리고 ' + string
        else:
            output.append([index, string])

    return [item[1] for item in output]

In [19]:
# Inference를 위한 test.csv 파일 로드
test = pd.read_csv('./test.csv')
test_to_list = test['질문'].tolist()
keywords = ['그리고', '또한,']  # 원하는 키워드 추가 가능
test_index, test_list = split_sentences_with_keywords(test_to_list, keywords)

In [30]:
# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
preds = []
file = open("prediction.txt", "w")
# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(test['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    # input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    # output_sequences = model.generate(
    #     input_ids=input_ids.to(device),
    #     max_length=300,
    #     temperature=0.9,
    #     top_k=1,
    #     top_p=0.9,
    #     repetition_penalty=1.2,
    #     do_sample=True,
    #     num_return_sequences=1
    # )
    # output_sequences =gen(test_question)
    gened = model.generate(
        **tokenizer(
            f"### 질문: {test_question}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=512,
        top_k =1,
        top_p=0.9,
        repetition_penalty=1.2,
        early_stopping=True,
        num_return_sequences =1,
        do_sample=True,
        eos_token_id=2,
    )

    # 생성된 텍스트(답변) 저장
    # for generated_sequence in output_sequences:
    # full_text = tokenizer.decode(gened[0])
    # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
    # answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
    # answer_only = full_text.split("답변:",1)[1].strip()
    # answer_only = answer_only.replace('\n', ' ')

    # preds.append(answer_only)
    # file.write(""test_question + " " +answer_only+ "\n")
# file.close()
 # 생성된 텍스트(답변) 저장
    for generated_sequence in gened:
      full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
      start_index = full_text.rfind("### 답변:") + len("### 답변:")  # 답변 시작 인덱스
      end_index = full_text.rfind("<|endoftext|")  # 답변 종료 인덱스
      answer = full_text[start_index:end_index].strip()  # 답변 추출 및 공백 제거
      preds.append(answer)

answers = merge_answers_by_index(test_index, preds)

  0%|          | 0/130 [00:00<?, ?it/s]


RuntimeError: "topk_cpu" not implemented for 'Half'

In [ ]:
len(answers)

In [ ]:
df = pd.DataFrame(answers, columns=['Column_Name'])  # You need to specify the column name(s)
df.to_csv('./polyglot_5.8b_preds.csv',index=False)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip install -U sentence-transformers

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

In [ ]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./polyglot_12.8b_finetuned_submit_3.csv', index=False)

In [ ]:
word = '답변:'

answer = word.split("for",1)[0].strip()
print(answer)